# Modeling Cineplex Concessions - Simple Version

- Stephen W. Thomas
- November 2020
- For: MMA/GMMA/MMAI 869

This Notebook will build a basic ML classifier model. Please feel free to use and modify as you see fit.

In [5]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Load in the data 

In [16]:
items = pd.read_csv("C:/Users/reall/OneDrive/Documents/Aarcha/Machine Learning and AI - MMA 869/clean_v2/items_v2.csv")

In [13]:
visits = pd.read_csv("C:/Users/reall/OneDrive/Documents/Aarcha/Machine Learning and AI - MMA 869/clean_v2/visits_v2.csv")

In [14]:
locations = pd.read_csv("C:/Users/reall/OneDrive/Documents/Aarcha/Machine Learning and AI - MMA 869/clean_v2/locations_v2.csv")

In [18]:
visits_items = pd.read_csv("C:/Users/reall/OneDrive/Documents/Aarcha/Machine Learning and AI - MMA 869/clean_v2/visit_items_v2.csv")

In [19]:
films = pd.read_csv("C:/Users/reall/OneDrive/Documents/Aarcha/Machine Learning and AI - MMA 869/clean_v2/films_v2.csv")

In [20]:
experience_type = pd.read_csv("C:/Users/reall/OneDrive/Documents/Aarcha/Machine Learning and AI - MMA 869/clean_v2/experience_type_v2.csv")

In [21]:
sales_channels = pd.read_csv("C:/Users/reall/OneDrive/Documents/Aarcha/Machine Learning and AI - MMA 869/clean_v2/sales_channels_v2.csv")

In [22]:
timeslice = pd.read_csv("C:/Users/reall/OneDrive/Documents/Aarcha/Machine Learning and AI - MMA 869/clean_v2/timeslice_v2.csv")

In [23]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.float_format', lambda x: '%.1f' % x):
    display(visits.head())

,Visit_ID,CINEPLEX_D_Membership_ID,Visit_Date,Visit_TimeSlice_ID,Location_ID,Auditorium_Experience_ID,Film_ID,Seating_Experience_ID,Sales_Channel_ID,Performance_Experience_ID,Performance_Type_ID,Spend,Revenue,Points_Earned,Points_Redeemed,Discount,Has_US_Is_LrgPopcorn,Has_US_Is_Butter,Has_US_Is_Alcohol,Has_US_Is_Candy,Has_US_Is_Coffee,Has_US_Is_Kiddietray,Has_US_Is_Voucher,Has_US_Is_MenuFood
0,2,23777,2019-01-01,15,7253,1,28757,1,8,1,1,15.0,15.0,100,0,1.7,0,0,0,0,0,0,0,0
1,3,60723,2019-01-01,15,7123,0,0,0,8,0,0,29.6,29.6,297,0,3.3,0,1,0,0,0,0,1,1
2,4,61080,2019-01-01,15,7290,2,24899,2,8,2,1,28.4,28.4,235,0,3.2,1,1,0,0,0,0,0,0
3,5,103538,2019-01-01,14,1142,1,24977,1,2,1,1,0.0,10.0,0,1000,0.0,0,0,0,0,0,0,0,0
4,6,120196,2019-01-01,23,9181,1,29694,1,2,1,1,17.6,17.6,100,0,1.9,0,0,0,0,0,0,0,0


# Basic Wrangling and Cleaning

- Merge all the fact tables (e.g., `films`, `locations`) into on big `visits` table - this table will act as the "historical truth"
- Fill in missing values with "Unknown"
- Extract some date info (e.g., like day of week, name of month, etc)


In [24]:
visits.shape

visits = visits.merge(films, how="left", on="Film_ID")
visits = visits.merge(experience_type, how="left")
visits = visits.merge(sales_channels, how="left")
visits = visits.merge(timeslice, left_on="Visit_TimeSlice_ID", right_on="TimeSlice_ID", how="left")
visits = visits.merge(locations, how="left")

visits.shape

(36258, 24)

(36258, 60)

In [25]:
# Fill in empty values on language
visits['Market_Language'] = visits['Market_Language'].fillna('Unknown')

In [26]:
# Extract some info about the date
visits['Visit_Date'] = pd.to_datetime(visits['Visit_Date'])
visits['Visit_Date_DOW'] = visits['Visit_Date'].dt.day_name()
visits['Visit_Date_Week'] = visits['Visit_Date'].dt.strftime('%U')
visits['Visit_Date_Month'] = visits['Visit_Date'].dt.month_name()
visits['Visit_Date_IsWeekend'] = np.where(visits['Visit_Date'].dt.weekday < 5, 0, 1)

In [27]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.float_format', lambda x: '%.1f' % x):
    display(visits.head())

,Visit_ID,CINEPLEX_D_Membership_ID,Visit_Date,Visit_TimeSlice_ID,Location_ID,Auditorium_Experience_ID,Film_ID,Seating_Experience_ID,Sales_Channel_ID,Performance_Experience_ID,Performance_Type_ID,Spend,Revenue,Points_Earned,Points_Redeemed,Discount,Has_US_Is_LrgPopcorn,Has_US_Is_Butter,Has_US_Is_Alcohol,Has_US_Is_Candy,Has_US_Is_Coffee,Has_US_Is_Kiddietray,Has_US_Is_Voucher,Has_US_Is_MenuFood,Film_Title,Title_ID,Title_Name,Theatrical_Release_Date,Film_Runtime,Language,Market_Language,Release_Pattern,Performance_Type,Is_IMAX,Is_DBOX_Capable,Release_Status,Media_Class_Description,Is_4Dx,Genre_Hierarchy_1,Auditorium_Experience_Description,Is_Premium,Sales_Channel,TimeSlice_ID,TimeSlice_Description,TimeSlice_Start_Time,TimeSlice_End_Time,TimeSlice_Hour_12,TimeSlice_Hour_24,Location_Name,Location_Short_Name,Location_Interactive_Name,Address1,City,Province_CD,Postal_Code,Country_CD,Is_VIP,Is_Outtakes,Is_Licensed_Bar,Is_Licensed_Lounge,Visit_Date_DOW,Visit_Date_Week,Visit_Date_Month,Visit_Date_IsWeekend
0,2,23777,2019-01-01,15,7253,1,28757,1,8,1,1,15.0,15.0,100,0,1.7,0,0,0,0,0,0,0,0,Vice,23956.0,Vice,12/25/2018,133.0,English,English,Wide,Film Presentation,0,0,A,Adult,0.0,Drama,Regular,0,Point of Sale,15,1 - Early Matinee,00:00.0,30:00.0,1:00 PM,13,CPX Eglinton T.C.,Warden&Eglin,Cineplex Odeon Eglinton Town Centre Cinemas,22 Lebovic Avenue,Toronto,ON,M1L 4V9,CA,0,1,1,0,Tuesday,00,January,0
1,3,60723,2019-01-01,15,7123,0,0,0,8,0,0,29.6,29.6,297,0,3.3,0,1,0,0,0,0,1,1,<None>,0.0,<None>,1/1/1900,0.0,<None>,<None>,<None>,<None>,0,0,B,None,0.0,Unknown,<Unknown>,0,Point of Sale,15,1 - Early Matinee,00:00.0,30:00.0,1:00 PM,13,CPX Winston Churchill VIP,Winston Churchill,Cineplex Cinemas Winston Churchill & VIP,2081 Winston Park Dr.,Oakville,ON,L6H 6P5,CA,1,1,1,0,Tuesday,00,January,0
2,4,61080,2019-01-01,15,7290,2,24899,2,8,2,1,28.4,28.4,235,0,3.2,1,1,0,0,0,0,0,0,Aquaman,22133.0,Aquaman,12/21/2018,143.0,English,English,Wide,Film Presentation,0,1,A,Adult,0.0,Adaptation,AVX,1,Point of Sale,15,1 - Early Matinee,00:00.0,30:00.0,1:00 PM,13,CPX Hamilton Mountain,CPX Hamilton,Cineplex Cinemas Hamilton Mountain,795 Paramount Drive,Stoney Creek,ON,L8J 0B4,CA,0,1,1,0,Tuesday,00,January,0
3,5,103538,2019-01-01,14,1142,1,24977,1,2,1,1,0.0,10.0,0,1000,0.0,0,0,0,0,0,0,0,0,Mary Poppins Returns,22187.0,Mary Poppins Returns,12/19/2018,131.0,English,English,Wide,Film Presentation,0,1,A,Family,0.0,Comedy,Regular,0,Kiosk,14,1 - Early Matinee,30:00.0,00:00.0,12:00 PM,12,CPX Pitt Meadows,Meadowtown,Cineplex Odeon Meadowtown Cinemas,"#410, 19800 Lougheed Highway",Pitt Meadows,BC,V3Y 2W1,CA,0,1,0,0,Tuesday,00,January,0
4,6,120196,2019-01-01,23,9181,1,29694,1,2,1,1,17.6,17.6,100,0,1.9,0,0,0,0,0,0,0,0,Deuxime acte,23976.0,Second Act,12/21/2018,104.0,French,French,Wide,Film Presentation,0,0,A,Adult,0.0,Comedy,Regular,0,Kiosk,23,2 - Late Matinee,00:00.0,30:00.0,5:00 PM,17,CPX Beauport,Beauport,Cinma Cineplex Odeon Beauport,825 rue Clmenceau,Beauport,QC,G1C 2K6,CA,0,0,0,0,Tuesday,00,January,0


In [28]:
visits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36258 entries, 0 to 36257
Data columns (total 64 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Visit_ID                           36258 non-null  int64         
 1   CINEPLEX_D_Membership_ID           36258 non-null  int64         
 2   Visit_Date                         36258 non-null  datetime64[ns]
 3   Visit_TimeSlice_ID                 36258 non-null  int64         
 4   Location_ID                        36258 non-null  int64         
 5   Auditorium_Experience_ID           36258 non-null  int64         
 6   Film_ID                            36258 non-null  int64         
 7   Seating_Experience_ID              36258 non-null  int64         
 8   Sales_Channel_ID                   36258 non-null  int64         
 9   Performance_Experience_ID          36258 non-null  int64         
 10  Performance_Type_ID               

In [29]:
# Need to remove everything we know about this visit

# `df` will act as the main training data. 
# Think of `df` as "what will be available during prediction time" and `visits` as `historical information from which we can engineer
# features. 
# This `df` will be the same as visits, except with columsn removed that either (a) would not be available
X = visits.copy()


remove_cols = [

            # Can't have these (won't be available in future)
           'Spend', 'Revenue', 'Points_Earned', 'Points_Redeemed', 'Discount',
           'Has_US_Is_LrgPopcorn', 'Has_US_Is_Butter', 'Has_US_Is_Alcohol', 'Has_US_Is_Candy',
           'Has_US_Is_Coffee', 'Has_US_Is_Kiddietray', 'Has_US_Is_Voucher', 'Has_US_Is_MenuFood',
    
            # Don't want these (won't be helpful, I don't think.)
           'TimeSlice_Description', 'TimeSlice_Start_Time', 'TimeSlice_End_Time','TimeSlice_Hour_12', 'TimeSlice_Hour_24',
           'Film_Title', 'Title_Name', 'Theatrical_Release_Date', 'Release_Status', 'Release_Pattern', 'Title_ID', 
           'Location_Name', 'Location_Short_Name', 'Location_Interactive_Name',
           'Address1', 'City', 'Postal_Code',
          ] 

X = X.drop(remove_cols, axis=1)


In [30]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36258 entries, 0 to 36257
Data columns (total 34 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Visit_ID                           36258 non-null  int64         
 1   CINEPLEX_D_Membership_ID           36258 non-null  int64         
 2   Visit_Date                         36258 non-null  datetime64[ns]
 3   Visit_TimeSlice_ID                 36258 non-null  int64         
 4   Location_ID                        36258 non-null  int64         
 5   Auditorium_Experience_ID           36258 non-null  int64         
 6   Film_ID                            36258 non-null  int64         
 7   Seating_Experience_ID              36258 non-null  int64         
 8   Sales_Channel_ID                   36258 non-null  int64         
 9   Performance_Experience_ID          36258 non-null  int64         
 10  Performance_Type_ID               

# Training/Test Split

In [31]:
# The name of the column in visits that has the target value we want to predict
# Change this to whatever you need/want!
target_col = 'Has_US_Is_Kiddietray'

y = visits[target_col]

train_index = X['Visit_ID'] <= 88863
test_index = ~train_index


visits_train, visits_test = visits[train_index], visits[test_index]

visits_train.shape
visits_test.shape

X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

X_train.shape
y_train.shape

X_test.shape
y_test.shape

(31079, 64)

(5179, 64)

(31079, 34)

(31079,)

(5179, 34)

(5179,)

# Feature Engineering (On the Training Data Only)

We need to take special care avoid data leakage. In particular, when we are engineering features that involves calculations of the data, those calculations should be trained (fit) from the training data.  

## Per-Member Aggregate Features

We now calcualte member-level aggregation statistics, such as:
 - The number of total visits
 - The number of differnet locations the member has visited
 - The number of times a member has purchased large popcorn
 - The number of times a member has purchased butter
 - etc.

In [32]:
# Define my own version of the 'mode' function, which will just return the first mode (in case of tie) and 
# thus avoid an annoying error from groupby (whcih always wants just one value)
my_mode = lambda x: pd.Series.mode(x)[0]

# Use a simple group by to compute some aggregrate statistics. We can do anything we want here,
# this is just an example!
members_features = visits_train.groupby('CINEPLEX_D_Membership_ID').agg(
{
    'Visit_Date': ['count'],

    'Film_ID': [pd.Series.nunique, my_mode],
    'Film_Runtime': ['mean', 'max'],
    'Is_VIP': ['mean', 'max'],
    'Market_Language': [my_mode],
    'Genre_Hierarchy_1': [pd.Series.nunique, my_mode],

    'Spend': ['min', 'mean', 'max', 'sum'],
    'Discount': ['mean'],

    'Has_US_Is_LrgPopcorn': ['count', 'sum', 'mean', 'max'],
    'Has_US_Is_Butter': ['count', 'sum', 'mean', 'max'],
    'Has_US_Is_Alcohol': ['count', 'sum', 'mean', 'max'],
    'Has_US_Is_Candy': ['count', 'sum', 'mean', 'max'],
    'Has_US_Is_Kiddietray': ['count', 'sum', 'mean', 'max'],
    'Has_US_Is_Voucher': ['count', 'sum', 'mean', 'max'],
    'Has_US_Is_MenuFood': ['count', 'sum', 'mean', 'max'],
    'Has_US_Is_Coffee': ['count', 'sum', 'mean', 'max'],
}).reset_index()


# Pandas group-by creates a MultiIndex, which we don't want. The following few lines
# will rename the columns of the dataframe to something more reasonbale.
members_features.columns = ["_".join(x) for x in members_features.columns.ravel()]
members_features = members_features.rename(columns={'CINEPLEX_D_Membership_ID_': "CINEPLEX_D_Membership_ID"})
members_features.columns = members_features.columns.str.replace("<lambda_0>", "mode")
members_features.columns = members_features.columns.str.replace("<lambda>", "mode")

# We want to make sure we have all numeric data, so let's do OHE (dummies) to be sure.
members_features = pd.get_dummies(members_features)

In [33]:
# Let's take a peek
members_features.shape
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.float_format', lambda x: '%.1f' % x):
    members_features.head()

(5778, 64)

,CINEPLEX_D_Membership_ID,Visit_Date_count,Film_ID_nunique,Film_ID_mode,Film_Runtime_mean,Film_Runtime_max,Is_VIP_mean,Is_VIP_max,Genre_Hierarchy_1_nunique,Spend_min,Spend_mean,Spend_max,Spend_sum,Discount_mean,Has_US_Is_LrgPopcorn_count,Has_US_Is_LrgPopcorn_sum,Has_US_Is_LrgPopcorn_mean,Has_US_Is_LrgPopcorn_max,Has_US_Is_Butter_count,Has_US_Is_Butter_sum,Has_US_Is_Butter_mean,Has_US_Is_Butter_max,Has_US_Is_Alcohol_count,Has_US_Is_Alcohol_sum,Has_US_Is_Alcohol_mean,Has_US_Is_Alcohol_max,Has_US_Is_Candy_count,Has_US_Is_Candy_sum,Has_US_Is_Candy_mean,Has_US_Is_Candy_max,Has_US_Is_Kiddietray_count,Has_US_Is_Kiddietray_sum,Has_US_Is_Kiddietray_mean,Has_US_Is_Kiddietray_max,Has_US_Is_Voucher_count,Has_US_Is_Voucher_sum,Has_US_Is_Voucher_mean,Has_US_Is_Voucher_max,Has_US_Is_MenuFood_count,Has_US_Is_MenuFood_sum,Has_US_Is_MenuFood_mean,Has_US_Is_MenuFood_max,Has_US_Is_Coffee_count,Has_US_Is_Coffee_sum,Has_US_Is_Coffee_mean,Has_US_Is_Coffee_max,Market_Language_mode_<None>,Market_Language_mode_English,Market_Language_mode_French,Genre_Hierarchy_1_mode_Action,Genre_Hierarchy_1_mode_Adaptation,Genre_Hierarchy_1_mode_Adventure,Genre_Hierarchy_1_mode_Comedy,Genre_Hierarchy_1_mode_Documentary,Genre_Hierarchy_1_mode_Drama,Genre_Hierarchy_1_mode_Family,Genre_Hierarchy_1_mode_Fantasy,Genre_Hierarchy_1_mode_Foreign,Genre_Hierarchy_1_mode_Horror,Genre_Hierarchy_1_mode_Musical,Genre_Hierarchy_1_mode_Sequel,Genre_Hierarchy_1_mode_Suspense,Genre_Hierarchy_1_mode_Thriller,Genre_Hierarchy_1_mode_Unknown
0,3752,2,2,0,46.5,93.0,0.0,0,2,0.0,6.3,12.7,12.7,0.7,2,0,0.0,0,2,1,0.5,1,2,0,0.0,0,2,0,0.0,0,2,0,0.0,0,2,0,0.0,0,2,0,0.0,0,2,0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,5052,2,2,27465,89.0,90.0,0.0,0,1,54.2,69.2,84.2,138.3,7.7,2,0,0.0,0,2,0,0.0,0,2,0,0.0,0,2,2,1.0,1,2,0,0.0,0,2,0,0.0,0,2,1,0.5,1,2,0,0.0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,5548,21,20,0,107.5,149.0,0.0,0,7,0.0,14.3,38.9,300.1,1.6,21,6,0.3,1,21,0,0.0,0,21,0,0.0,0,21,1,0.0,1,21,0,0.0,0,21,0,0.0,0,21,2,0.1,1,21,0,0.0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,7157,6,6,0,98.7,143.0,0.0,0,4,7.5,21.2,38.8,126.9,2.4,6,4,0.7,1,6,0,0.0,0,6,0,0.0,0,6,0,0.0,0,6,0,0.0,0,6,0,0.0,0,6,0,0.0,0,6,0,0.0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,15934,1,1,28848,103.0,103.0,0.0,0,1,18.9,18.9,18.9,18.9,2.1,1,1,1.0,1,1,1,1.0,1,1,0,0.0,0,1,0,0.0,0,1,0,0.0,0,1,0,0.0,0,1,0,0.0,0,1,0,0.0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


## One Hot Encoding of Categorical Features

In [34]:
from sklearn.preprocessing import OneHotEncoder

# Which columns are categorical (that we want to encode)?
cat_feature_names = ['Language', 'Market_Language', 'Performance_Type',
       'Media_Class_Description', 'Visit_Date_DOW', 'Visit_Date_Month',
       'Genre_Hierarchy_1', 'Auditorium_Experience_Description',
       'Sales_Channel', 'Province_CD', 'Country_CD']
print(cat_feature_names)

# Fit the OneHotEncoder and save the `ohe` object for later.
ohe = OneHotEncoder(categories='auto', handle_unknown='ignore', sparse=False)
ohe = ohe.fit(X_train[cat_feature_names])
ohe_feature_names = list(ohe.get_feature_names(cat_feature_names))

print("\n\nNames of {} OHE features:".format(len(ohe_feature_names)))
print(ohe_feature_names)

['Language', 'Market_Language', 'Performance_Type', 'Media_Class_Description', 'Visit_Date_DOW', 'Visit_Date_Month', 'Genre_Hierarchy_1', 'Auditorium_Experience_Description', 'Sales_Channel', 'Province_CD', 'Country_CD']


Names of 103 OHE features:
['Language_<None>', 'Language_Arabic', 'Language_Cantonese', 'Language_Egyptian', 'Language_English', 'Language_Filipino', 'Language_French', 'Language_German', 'Language_Gujarati', 'Language_Hebrew', 'Language_Hindi', 'Language_Italian', 'Language_Japanese', 'Language_Korean', 'Language_Lebanese', 'Language_Mandarin', 'Language_Polish', 'Language_Punjabi', 'Language_Spanish', 'Language_Telugu', 'Language_Thai', 'Language_Urdu', 'Language_Vietnamese', 'Market_Language_<None>', 'Market_Language_English', 'Market_Language_French', 'Performance_Type_<None>', 'Performance_Type_Alt Prog Film Event', 'Performance_Type_Concert', 'Performance_Type_Documentary', 'Performance_Type_FRCE Feature Release', 'Performance_Type_Film Presentation', 'Performa

## Main Function to Do Feature Transformations

Now that the feature engieering steps have been trained/fit, we can now extract features from the data.

The following function will take raw data (whehter it be training data or test dat) and do the following:
- Encode categorical features (using the OHE trained above)
- Look up the member's statistics from previous visits. (If member has not previous visits, averages/means are used.)
- Remove columns that aren't needed.

In [35]:
def get_features(X):
    # This function assumes that the following variables have already been assigned:
    # - cat_attrs and cat_feature_names
    # - members_features
    
    # Use the OHE to get numerical features from categorical
    ohe_features = ohe.transform(X[cat_feature_names])
    ohe_features_df = pd.DataFrame(ohe_features, columns=ohe_feature_names)

    # Look up member aggregation statistics
    X_features = X.merge(members_features, how="left", on="CINEPLEX_D_Membership_ID" ).reset_index()
    
    # For any members that didn't have agg features, then just fill them in with mean values.
    X_features = X_features.fillna(members_features.mean())
    
    # Remove features we don't want an more
    X_features = X_features.drop(cat_feature_names, axis=1)
    X_features = X_features.drop(['Visit_ID', 'Visit_Date', 'index'], axis=1)
    

    # Merge the numeric features and encoded features together.
    features_all = pd.concat([X_features, ohe_features_df], axis=1)

    
    return X_features

### Training

In [36]:
X_train_features = get_features(X_train)

# Let's take a peek
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.float_format', lambda x: '%.1f' % x):
    X_train_features.head()

,CINEPLEX_D_Membership_ID,Visit_TimeSlice_ID,Location_ID,Auditorium_Experience_ID,Film_ID,Seating_Experience_ID,Sales_Channel_ID,Performance_Experience_ID,Performance_Type_ID,Film_Runtime,Is_IMAX,Is_DBOX_Capable,Is_4Dx,Is_Premium,TimeSlice_ID,Is_VIP,Is_Outtakes,Is_Licensed_Bar,Is_Licensed_Lounge,Visit_Date_Week,Visit_Date_IsWeekend,Visit_Date_count,Film_ID_nunique,Film_ID_mode,Film_Runtime_mean,Film_Runtime_max,Is_VIP_mean,Is_VIP_max,Genre_Hierarchy_1_nunique,Spend_min,Spend_mean,Spend_max,Spend_sum,Discount_mean,Has_US_Is_LrgPopcorn_count,Has_US_Is_LrgPopcorn_sum,Has_US_Is_LrgPopcorn_mean,Has_US_Is_LrgPopcorn_max,Has_US_Is_Butter_count,Has_US_Is_Butter_sum,Has_US_Is_Butter_mean,Has_US_Is_Butter_max,Has_US_Is_Alcohol_count,Has_US_Is_Alcohol_sum,Has_US_Is_Alcohol_mean,Has_US_Is_Alcohol_max,Has_US_Is_Candy_count,Has_US_Is_Candy_sum,Has_US_Is_Candy_mean,Has_US_Is_Candy_max,Has_US_Is_Kiddietray_count,Has_US_Is_Kiddietray_sum,Has_US_Is_Kiddietray_mean,Has_US_Is_Kiddietray_max,Has_US_Is_Voucher_count,Has_US_Is_Voucher_sum,Has_US_Is_Voucher_mean,Has_US_Is_Voucher_max,Has_US_Is_MenuFood_count,Has_US_Is_MenuFood_sum,Has_US_Is_MenuFood_mean,Has_US_Is_MenuFood_max,Has_US_Is_Coffee_count,Has_US_Is_Coffee_sum,Has_US_Is_Coffee_mean,Has_US_Is_Coffee_max,Market_Language_mode_<None>,Market_Language_mode_English,Market_Language_mode_French,Genre_Hierarchy_1_mode_Action,Genre_Hierarchy_1_mode_Adaptation,Genre_Hierarchy_1_mode_Adventure,Genre_Hierarchy_1_mode_Comedy,Genre_Hierarchy_1_mode_Documentary,Genre_Hierarchy_1_mode_Drama,Genre_Hierarchy_1_mode_Family,Genre_Hierarchy_1_mode_Fantasy,Genre_Hierarchy_1_mode_Foreign,Genre_Hierarchy_1_mode_Horror,Genre_Hierarchy_1_mode_Musical,Genre_Hierarchy_1_mode_Sequel,Genre_Hierarchy_1_mode_Suspense,Genre_Hierarchy_1_mode_Thriller,Genre_Hierarchy_1_mode_Unknown
0,23777,15,7253,1,28757,1,8,1,1,133.0,0,0,0.0,0,15,0,1,1,0,00,0,6,6,27641,122.2,133.0,0.0,0,1,8.8,16.8,24.2,101.1,1.9,6,0,0.0,0,6,0,0.0,0,6,0,0.0,0,6,0,0.0,0,6,0,0.0,0,6,0,0.0,0,6,0,0.0,0,6,0,0.0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,60723,15,7123,0,0,0,8,0,0,0.0,0,0,0.0,0,15,1,1,1,0,00,0,12,7,0,72.8,169.0,0.4,1,5,0.0,29.4,50.0,352.4,3.3,12,2,0.2,1,12,2,0.2,1,12,1,0.1,1,12,1,0.1,1,12,0,0.0,0,12,1,0.1,1,12,9,0.8,1,12,0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,61080,15,7290,2,24899,2,8,2,1,143.0,0,1,0.0,1,15,0,1,1,0,00,0,6,6,24899,113.7,143.0,0.0,0,3,0.0,18.3,63.5,109.8,2.0,6,1,0.2,1,6,2,0.3,1,6,0,0.0,0,6,1,0.2,1,6,1,0.2,1,6,0,0.0,0,6,0,0.0,0,6,0,0.0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,103538,14,1142,1,24977,1,2,1,1,131.0,0,1,0.0,0,14,0,1,0,0,00,0,3,3,24758,122.0,131.0,0.0,0,3,0.0,5.6,16.7,16.7,0.6,3,0,0.0,0,3,0,0.0,0,3,0,0.0,0,3,0,0.0,0,3,0,0.0,0,3,0,0.0,0,3,0,0.0,0,3,0,0.0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,120196,23,9181,1,29694,1,2,1,1,104.0,0,0,0.0,0,23,0,0,0,0,00,0,10,8,28491,121.9,143.0,0.0,0,5,10.2,15.1,27.6,150.8,1.7,10,0,0.0,0,10,0,0.0,0,10,0,0.0,0,10,0,0.0,0,10,0,0.0,0,10,0,0.0,0,10,2,0.2,1,10,0,0.0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


### Testing

In [37]:
X_test_features = get_features(X_test)

# Let's take a peek
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.float_format', lambda x: '%.1f' % x):
    X_test_features.head()

,CINEPLEX_D_Membership_ID,Visit_TimeSlice_ID,Location_ID,Auditorium_Experience_ID,Film_ID,Seating_Experience_ID,Sales_Channel_ID,Performance_Experience_ID,Performance_Type_ID,Film_Runtime,Is_IMAX,Is_DBOX_Capable,Is_4Dx,Is_Premium,TimeSlice_ID,Is_VIP,Is_Outtakes,Is_Licensed_Bar,Is_Licensed_Lounge,Visit_Date_Week,Visit_Date_IsWeekend,Visit_Date_count,Film_ID_nunique,Film_ID_mode,Film_Runtime_mean,Film_Runtime_max,Is_VIP_mean,Is_VIP_max,Genre_Hierarchy_1_nunique,Spend_min,Spend_mean,Spend_max,Spend_sum,Discount_mean,Has_US_Is_LrgPopcorn_count,Has_US_Is_LrgPopcorn_sum,Has_US_Is_LrgPopcorn_mean,Has_US_Is_LrgPopcorn_max,Has_US_Is_Butter_count,Has_US_Is_Butter_sum,Has_US_Is_Butter_mean,Has_US_Is_Butter_max,Has_US_Is_Alcohol_count,Has_US_Is_Alcohol_sum,Has_US_Is_Alcohol_mean,Has_US_Is_Alcohol_max,Has_US_Is_Candy_count,Has_US_Is_Candy_sum,Has_US_Is_Candy_mean,Has_US_Is_Candy_max,Has_US_Is_Kiddietray_count,Has_US_Is_Kiddietray_sum,Has_US_Is_Kiddietray_mean,Has_US_Is_Kiddietray_max,Has_US_Is_Voucher_count,Has_US_Is_Voucher_sum,Has_US_Is_Voucher_mean,Has_US_Is_Voucher_max,Has_US_Is_MenuFood_count,Has_US_Is_MenuFood_sum,Has_US_Is_MenuFood_mean,Has_US_Is_MenuFood_max,Has_US_Is_Coffee_count,Has_US_Is_Coffee_sum,Has_US_Is_Coffee_mean,Has_US_Is_Coffee_max,Market_Language_mode_<None>,Market_Language_mode_English,Market_Language_mode_French,Genre_Hierarchy_1_mode_Action,Genre_Hierarchy_1_mode_Adaptation,Genre_Hierarchy_1_mode_Adventure,Genre_Hierarchy_1_mode_Comedy,Genre_Hierarchy_1_mode_Documentary,Genre_Hierarchy_1_mode_Drama,Genre_Hierarchy_1_mode_Family,Genre_Hierarchy_1_mode_Fantasy,Genre_Hierarchy_1_mode_Foreign,Genre_Hierarchy_1_mode_Horror,Genre_Hierarchy_1_mode_Musical,Genre_Hierarchy_1_mode_Sequel,Genre_Hierarchy_1_mode_Suspense,Genre_Hierarchy_1_mode_Thriller,Genre_Hierarchy_1_mode_Unknown
0,4025361,20,7411,0,0,0,8,0,0,0.0,0,0,0.0,0,20,0,1,0,0,44,1,5.4,4.9,15469.8,106.7,146.1,0.2,0.3,3.0,15.3,29.4,48.7,145.7,3.3,5.4,1.1,0.2,0.5,5.4,0.9,0.2,0.4,5.4,0.1,0.0,0.0,5.4,1.3,0.3,0.6,5.4,0.2,0.1,0.1,5.4,0.2,0.0,0.1,5.4,0.6,0.1,0.3,5.4,0.1,0.0,0.1,0.1,0.8,0.0,0.4,0.2,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1
1,4101003,31,3144,2,28116,4,8,1,1,128.0,0,1,1.0,1,31,0,1,0,0,44,1,5.4,4.9,15469.8,106.7,146.1,0.2,0.3,3.0,15.3,29.4,48.7,145.7,3.3,5.4,1.1,0.2,0.5,5.4,0.9,0.2,0.4,5.4,0.1,0.0,0.0,5.4,1.3,0.3,0.6,5.4,0.2,0.1,0.1,5.4,0.2,0.0,0.1,5.4,0.6,0.1,0.3,5.4,0.1,0.0,0.1,0.1,0.8,0.0,0.4,0.2,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1
2,4168092,31,3132,2,30567,4,8,1,1,144.0,0,0,0.0,1,31,0,1,1,0,44,1,20.0,15.0,0.0,90.2,182.0,0.0,0.0,7.0,0.0,31.1,92.5,621.9,3.5,20.0,2.0,0.1,1.0,20.0,10.0,0.5,1.0,20.0,0.0,0.0,0.0,20.0,9.0,0.5,1.0,20.0,7.0,0.3,1.0,20.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4202569,27,7112,1,29562,1,8,1,1,122.0,0,0,0.0,0,27,1,1,1,1,44,1,6.0,6.0,0.0,107.2,182.0,0.2,1.0,5.0,17.5,29.6,47.3,177.8,3.3,6.0,0.0,0.0,0.0,6.0,2.0,0.3,1.0,6.0,2.0,0.3,1.0,6.0,1.0,0.2,1.0,6.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,6.0,1.0,0.2,1.0,6.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4275633,29,7400,4,28116,4,8,1,1,128.0,0,1,1.0,1,29,1,1,1,1,44,1,10.0,9.0,24899.0,126.8,182.0,0.9,1.0,5.0,33.3,54.0,81.1,540.2,6.0,10.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,10.0,3.0,0.3,1.0,10.0,2.0,0.2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# (Optional) Write the training/testing files out

For inspection, or use in other programs/tools

In [39]:
X_train_features.to_csv('C:/Users/reall/OneDrive/Documents/Aarcha/Machine Learning and AI - MMA 869/clean_v2/gen_X_train_features.csv', index=False)

In [40]:
y_train.to_csv('C:/Users/reall/OneDrive/Documents/Aarcha/Machine Learning and AI - MMA 869/clean_v2/gen_y_train.csv', index=False)

In [42]:
X_test_features.to_csv('C:/Users/reall/OneDrive/Documents/Aarcha/Machine Learning and AI - MMA 869/clean_v2/gen_X_test_features.csv', index=False)

In [43]:
y_test.to_csv('C:/Users/reall/OneDrive/Documents/Aarcha/Machine Learning and AI - MMA 869/clean_v2/gen_y_test.csv', index=False)

In [41]:
#X_train_features.to_csv('C:/Users/reall/OneDrive/Documents/Aarcha/Machine Learning and AI - MMA 869/clean_v2/gen_X_train_features.csv', index=False)
#y_train.to_csv('C:/Users/reall/OneDrive/Documents/Aarcha/Machine Learning and AI - MMA 869/clean_v2/gen_y_train.csv', index=False)
#X_test_features.to_csv('C:/Users/reall/OneDrive/Documents/Aarcha/Machine Learning and AI - MMA 869/clean_v2/gen_X_test_features.csv', index=False)
#y_test.to_csv('C:/Users/reall/OneDrive/Documents/Aarcha/Machine Learning and AI - MMA 869/clean_v2/gen_y_test.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'gen_v2/gen_y_test_v2.csv'

# Machine Learning Time! (Modeling)

## Training

In [19]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=10, class_weight="balanced_subsample", criterion="gini", random_state=0)
clf = clf.fit(X_train_features, y_train)

## Testing

In [20]:
from sklearn.metrics import classification_report, accuracy_score, cohen_kappa_score, f1_score, log_loss, confusion_matrix

y_pred = clf.predict(X_test_features)

# How did we do?
print("F1 Score   = {:.3f}".format(f1_score(y_test, y_pred, average="macro")))
print("Accuracy   = {:.3f}".format(accuracy_score(y_test, y_pred)))
print("Kappa      = {:.3f}".format(cohen_kappa_score(y_test, y_pred)))
print("Log Loss   = {:.3f}".format(log_loss(y_test, y_pred)))
print("\nConfusion Matrix:")
unique_label = np.unique([y_test, y_pred])
cmtx = pd.DataFrame(
confusion_matrix(y_test, y_pred, labels=unique_label), 
index=['true:{:}'.format(x) for x in unique_label], 
columns=['pred:{:}'.format(x) for x in unique_label])
print(cmtx)
print("\nClassification Report:")
print(classification_report(y_test, y_pred)) 

F1 Score   = 0.706
Accuracy   = 0.751
Kappa      = 0.425
Log Loss   = 8.610

Confusion Matrix:
        pred:0  pred:1
true:0    2952     992
true:1     299     936

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.75      0.82      3944
           1       0.49      0.76      0.59      1235

    accuracy                           0.75      5179
   macro avg       0.70      0.75      0.71      5179
weighted avg       0.81      0.75      0.77      5179



## Model Inference

What did our model "learn"? What features are most important, etc?

In [21]:
# Let's measure the importances of the features
feature_names = X_train_features.columns

imp = pd.DataFrame({'Feature': feature_names, 'Importance': clf.feature_importances_})
imp = imp.sort_values('Importance', ascending=False)
imp.head(20)

,Feature,Importance
48,Has_US_Is_Candy_mean,0.240737
47,Has_US_Is_Candy_sum,0.202286
49,Has_US_Is_Candy_max,0.148261
6,Sales_Channel_ID,0.099303
36,Has_US_Is_LrgPopcorn_mean,0.035647
30,Spend_mean,0.033541
33,Discount_mean,0.026187
29,Spend_min,0.017907
35,Has_US_Is_LrgPopcorn_sum,0.014561
37,Has_US_Is_LrgPopcorn_max,0.013868
